In [3]:
import sys
sys.path.insert(0,r"C:\Users\LENOVO\Downloads")
import numpy as np
import pandas as pd
import csv
from sklearn.model_selection import train_test_split
import class_report as cr
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, accuracy_score, auc, f1_score, log_loss
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier

In [4]:
## Lectura de datos     
base_noroccidente = pd.read_csv(r'C:\Users\LENOVO\Downloads\BASE_NOROCCIDENTE.txt', sep=r'¡', engine='python')
base_noroccidente.head()

## Eliminación de variables: 
base_noroccidente2 = base_noroccidente[[
        #'CUCONUSE', 
        'SESUSERV', 'SERVDESC', 'SESUSUSC', 'SESUFEIN','DEPADESC', 'SESUCUSA', 'SESUSAPE','SESUCICL', 'SUSCNITC', 
        #'VECTOR','SESUCATE', 'SESUSUCA','SESULOCA','SESUSAAN','V1', 'V2', 'V3', 'V4', 'V5', 'V6',#'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 
        'TIPIFICACION_CLIENTE', 'SUMA','CANTIDAD','CALIFICACION_VECTOR_SERVICIO',
       #'VECTOR_CUALITATIVO_CONTRATO',#'BANCO_1', 'DIA_PAGO_1', 'DIA_PAGO_2','DIA_PAGO_3', 'FECHA_SUSPENSION','RANGO_EDAD', 'GENERO',
        'PROM_SUSC', 'CANAL_ENVIO', 'SEGMENTACION', 'REGIONAL','ESTRATO_AGRUPADO','DEPARTAMENTO_AGR','ANTIGUEDAD_DIAS', 'CANAL_PAGO']]

In [5]:
# Vista previa
base_noroccidente2.head()

,SESUSERV,SERVDESC,SESUSUSC,SESUFEIN,DEPADESC,SESUCUSA,SESUSAPE,SESUCICL,SUSCNITC,TIPIFICACION_CLIENTE,...,CANTIDAD,CALIFICACION_VECTOR_SERVICIO,PROM_SUSC,CANAL_ENVIO,SEGMENTACION,REGIONAL,ESTRATO_AGRUPADO,DEPARTAMENTO_AGR,ANTIGUEDAD_DIAS,CANAL_PAGO
0,3307,INTERNET,17081538,16/01/2020,ANTIOQUIA,1,48308,7.0,1017138705,NUEVO,...,2.0,88.0,"29092,17",DIGITAL,Hogares,NOROCCIDENTE,4.0,ANTIOQUIA,5853.0,DIGITAL
1,3905,TELEVISIÓN,17081641,17/01/2020,ANTIOQUIA,1,56228,7.0,98580909,NUEVO,...,3.0,100.0,"46342,38",DIGITAL,Hogares,NOROCCIDENTE,4.0,ANTIOQUIA,123.0,FISICO
2,3307,INTERNET,17081646,17/01/2020,ANTIOQUIA,3,113758,7.0,1100691658,NUEVO,...,2.0,0.0,"28570,17",DIGITAL,Hogares,NOROCCIDENTE,4.0,ANTIOQUIA,123.0,SIN PAGO
3,3905,TELEVISIÓN,17081650,17/01/2020,ANTIOQUIA,1,56228,7.0,79942778,NUEVO,...,3.0,83.0,"46350,88",DIGITAL,Hogares,NOROCCIDENTE,4.0,ANTIOQUIA,123.0,FISICO
4,3307,INTERNET,17081670,17/01/2020,ANTIOQUIA,2,92318,7.0,1152219039,NUEVO,...,2.0,33.0,"26405,25",HIBRIDO,Hogares,NOROCCIDENTE,4.0,ANTIOQUIA,834.0,HIBRIDO


In [6]:
## Eliminacion de clientes con datos NA
base_noroccidente3 = base_noroccidente2.dropna()

In [7]:
## Group by por nit para calificacion
b4 = base_noroccidente3[['SUSCNITC','CALIFICACION_VECTOR_SERVICIO']]
group_nit = b4.groupby(['SUSCNITC'])['CALIFICACION_VECTOR_SERVICIO'].mean().reset_index()


## Join de la nueva calificacion
base_noroccidente4 = pd.merge(base_noroccidente3, group_nit, how='left', on=['SUSCNITC'])

## Prueba promedio
b5 = base_noroccidente4[['SUSCNITC','CALIFICACION_VECTOR_SERVICIO_x','CALIFICACION_VECTOR_SERVICIO_y']]

### Creacion de la etiqueta de clasificacion
base_noroccidente4['y'] = pd.cut(x=base_noroccidente4['CALIFICACION_VECTOR_SERVICIO_y'],
                            bins=[-1,50,76,100],
                            labels=['No pago','Pago inoportuno','Pago'])


In [8]:
dataTypeSeries = base_noroccidente4.dtypes
dataTypeSeries

cat = base_noroccidente4.select_dtypes(include=['object','category'])
cuanti = base_noroccidente4.select_dtypes(include=['int64','float64'])
  
cat = cat.apply(lambda col: LabelEncoder().fit_transform(col))

In [9]:
cuanti.corr(method = 'pearson')

,SESUSERV,SESUSUSC,SESUCUSA,SESUCICL,SUMA,CANTIDAD,CALIFICACION_VECTOR_SERVICIO_x,ESTRATO_AGRUPADO,ANTIGUEDAD_DIAS,CALIFICACION_VECTOR_SERVICIO_y
SESUSERV,1.000000,0.176347,-0.011013,0.141118,-0.076083,-0.071307,-0.070589,-0.011573,0.017951,-0.049803
SESUSUSC,0.176347,1.000000,0.026382,0.208014,-0.254946,-0.220296,-0.161922,0.200760,-0.317324,-0.149432
SESUCUSA,-0.011013,0.026382,1.000000,-0.047015,-0.378606,-0.123390,-0.483692,-0.059239,-0.060989,-0.483022
SESUCICL,0.141118,0.208014,-0.047015,1.000000,0.173631,0.203122,0.067383,-0.286176,0.278885,0.069979
SUMA,-0.076083,-0.254946,-0.378606,0.173631,1.000000,0.888301,0.722265,0.026601,0.201659,0.681216
CANTIDAD,-0.071307,-0.220296,-0.123390,0.203122,0.888301,1.000000,0.455380,-0.003987,0.166769,0.403937
CALIFICACION_VECTOR_SERVICIO_x,-0.070589,-0.161922,-0.483692,0.067383,0.722265,0.455380,1.000000,0.056385,0.141966,0.929434
ESTRATO_AGRUPADO,-0.011573,0.200760,-0.059239,-0.286176,0.026601,-0.003987,0.056385,1.000000,-0.360022,0.060666
ANTIGUEDAD_DIAS,0.017951,-0.317324,-0.060989,0.278885,0.201659,0.166769,0.141966,-0.360022,1.000000,0.152745
CALIFICACION_VECTOR_SERVICIO_y,-0.049803,-0.149432,-0.483022,0.069979,0.681216,0.403937,0.929434,0.060666,0.152745,1.000000


In [10]:
base_noroccidente5=cuanti.join(cat)
print(base_noroccidente5.dtypes)

#base_noroccidente5.to_csv(r'C:\Users\pc\Desktop\base_sur_process.csv')

#base_noroccidente5 = pd.read_csv(r'C:\Users\pc\Desktop\base_sur_process.csv', sep=',', engine='python')

X = base_noroccidente5[[
        #'CUCONUSE', 
        'SESUSERV', 'SERVDESC', 
        #'SESUSUSC',
        'DEPADESC', 'SESUCUSA', 'SESUSAPE','SESUCICL', 'TIPIFICACION_CLIENTE', 
        #'SESUFEIN','SUSCNITC', 'VECTOR','SESUCATE', 'SESUSUCA','SESULOCA','SESUSAAN','RANGO_EDAD', 'GENERO', 
        #'V1', 'V2', 'V3', 'V4', 'V5', 'V6','V7', 'V8', 'V9', 'V10', 'V11', 'V12',
        #'REGIONAL',
        # 'SUMA',
        #'CANTIDAD',
    #'CALIFICACION_VECTOR_SERVICIO_x','CALIFICACION_VECTOR_SERVICIO_y','DEPARTAMENTO_AGR'
        #'VECTOR_CUALITATIVO_CONTRATO','BANCO_1', 'DIA_PAGO_1', 'DIA_PAGO_2','DIA_PAGO_3', 'FECHA_SUSPENSION', 
        'PROM_SUSC', 'CANAL_ENVIO',
       'SEGMENTACION', 'ESTRATO_AGRUPADO','ANTIGUEDAD_DIAS', 'CANAL_PAGO']]

y = base_noroccidente5['y']

SESUSERV                            int64
SESUSUSC                            int64
SESUCUSA                            int64
SESUCICL                          float64
SUMA                              float64
CANTIDAD                          float64
CALIFICACION_VECTOR_SERVICIO_x    float64
ESTRATO_AGRUPADO                  float64
ANTIGUEDAD_DIAS                   float64
CALIFICACION_VECTOR_SERVICIO_y    float64
SERVDESC                            int32
SESUFEIN                            int32
DEPADESC                            int32
SESUSAPE                            int32
SUSCNITC                            int32
TIPIFICACION_CLIENTE                int32
PROM_SUSC                           int32
CANAL_ENVIO                         int32
SEGMENTACION                        int32
REGIONAL                            int32
DEPARTAMENTO_AGR                    int32
CANAL_PAGO                          int32
y                                   int32
dtype: object


In [11]:
## Vista previa de las variables preliminares para clasificacion
X.head()

,SESUSERV,SERVDESC,DEPADESC,SESUCUSA,SESUSAPE,SESUCICL,TIPIFICACION_CLIENTE,PROM_SUSC,CANAL_ENVIO,SEGMENTACION,ESTRATO_AGRUPADO,ANTIGUEDAD_DIAS,CANAL_PAGO
0,3307,25,0,1,112062,7.0,1,129287,0,1,4.0,5853.0,0
1,3905,68,0,1,120871,7.0,1,470093,0,1,4.0,123.0,1
2,3307,25,0,3,12293,7.0,1,122432,0,1,4.0,123.0,4
3,3905,68,0,1,120871,7.0,1,470254,0,1,4.0,123.0,1
4,3307,25,0,2,157266,7.0,1,96144,2,1,4.0,834.0,2


In [12]:
from datetime import datetime
instanteInicial = datetime.now()
X_ = StandardScaler().fit_transform(X)
#X_ = X
X_train, X_test, y_train, y_test = train_test_split(X_, y, test_size=0.2,random_state=321)

classifiers = [
    KNeighborsClassifier(3),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1, max_iter=500),
    LogisticRegression(multi_class="multinomial", solver="lbfgs", C=5),
    GaussianNB()
    ]

# Competencia de clasificadores
log_cols=["Classifier", "Accuracy", "Log Loss", "F1_SCORE"]
log = pd.DataFrame(columns=log_cols)

for clf in classifiers:
    clf.fit(X_train, y_train)
    name = clf.__class__.__name__  
    print("="*30)
    print(name)
    print('****Resultados****')    
    train_predictions = clf.predict_proba(X_test)
    acc = cr.class_report(y_true = y_test, 
                                y_pred = clf.predict(X_test),
                                y_score = train_predictions)
    print("Métricas: {}".format(acc))
    log_entry = pd.DataFrame([[name, acc]])
    log = log.append(log_entry)
    
print("="*30)

KNeighborsClassifier
****Resultados****
Métricas:              precision    recall  f1-score   support      pred       AUC
0             0.721645  0.714717  0.718164   30829.0   30533.0  0.906938
1             0.890293  0.917596  0.903739  346234.0  356852.0  0.874663
2             0.651495  0.585355  0.616657  101675.0   91353.0  0.830082
avg / total   0.828717  0.833970  0.830817  478738.0  478738.0  0.927529
DecisionTreeClassifier
****Resultados****
Métricas:              precision    recall  f1-score   support      pred       AUC
0             0.852050  0.620941  0.718365   30829.0   22467.0  0.931033
1             0.837902  0.964640  0.896815  346234.0  398604.0  0.882671
2             0.697990  0.395879  0.505215  101675.0   57667.0  0.849055
avg / total   0.809098  0.821713  0.802155  478738.0  478738.0  0.945309
RandomForestClassifier
****Resultados****
Métricas:              precision    recall  f1-score   support      pred       AUC
0             0.978486  0.190308  0.318642 

In [13]:
# Importancia de variables
model = DecisionTreeClassifier(max_depth=5).fit(X_train, y_train)

importance = model.feature_importances_
# Total importancia de variables
for i,v in enumerate(importance):
	print('Variable: %0d, Score: %.5f' % (i,v))

Variable: 0, Score: 0.00000
Variable: 1, Score: 0.00000
Variable: 2, Score: 0.00000
Variable: 3, Score: 0.60163
Variable: 4, Score: 0.09843
Variable: 5, Score: 0.02106
Variable: 6, Score: 0.00805
Variable: 7, Score: 0.00000
Variable: 8, Score: 0.00000
Variable: 9, Score: 0.00227
Variable: 10, Score: 0.00000
Variable: 11, Score: 0.19723
Variable: 12, Score: 0.07133


In [ ]:
## El mejor modelo de clasificacion fue el KNN

In [14]:
classifiers = [
    KNeighborsClassifier(2),
    KNeighborsClassifier(4),
    KNeighborsClassifier(5),
    ]

# Competencia de clasificadores
log_cols=["Classifier", "Accuracy", "Log Loss", "F1_SCORE"]
log = pd.DataFrame(columns=log_cols)

for clf in classifiers:
    clf.fit(X_train, y_train)
    name = clf.__class__.__name__  
    print("="*30)
    print(name)
    print('****Resultados****')    
    train_predictions = clf.predict_proba(X_test)
    acc = cr.class_report(y_true = y_test, 
                                y_pred = clf.predict(X_test),
                                y_score = train_predictions)
    print("Métricas: {}".format(acc))
    log_entry = pd.DataFrame([[name, acc]])
    log = log.append(log_entry)
    
print("="*30)

KNeighborsClassifier
****Resultados****
Métricas:              precision    recall  f1-score   support      pred       AUC
0             0.627994  0.796069  0.702113   30829.0   39080.0  0.889323
1             0.859670  0.947573  0.901484  346234.0  381637.0  0.853744
2             0.716827  0.409058  0.520877  101675.0   58021.0  0.803886
avg / total   0.814414  0.823446  0.807811  478738.0  478738.0  0.910470
KNeighborsClassifier
****Resultados****
Métricas:              precision    recall  f1-score   support      pred       AUC
0             0.750065  0.745078  0.747563   30829.0   30624.0  0.917688
1             0.871766  0.948128  0.908345  346234.0  376562.0  0.885729
2             0.706353  0.497084  0.583523  101675.0   71552.0  0.844372
avg / total   0.828798  0.839259  0.829005  478738.0  478738.0  0.936099
KNeighborsClassifier
****Resultados****
Métricas:              precision    recall  f1-score   support      pred       AUC
0             0.785500  0.706413  0.743860   30

In [16]:
kf = StratifiedKFold(n_splits=5,shuffle=False)

summary_model = []
# Iteraciones del entrenamiento sobre los kfolds
#X_fin = X[['SESUSERV','SESUCUSA', 'SESUSAPE','SESUCICL', 'TIPIFICACION_CLIENTE', 'PROM_SUSC',
#'CANAL_ENVIO','SEGMENTACION', 'ESTRATO_AGRUPADO','ANTIGUEDAD_DIAS', 'CANAL_PAGO']]
X_fin = X
for train_index, test_index in kf.split(X_fin, y) :
    # Particion train test para validacion cruzada
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    # Entrenamiento de la red MLP
    model_final = KNeighborsClassifier(5).fit(X_train, y_train) 
    summary_model.append(cr.class_report(y_true = y_test, 
                                       y_pred = model_final.predict(X_test),
                                      y_score = model_final.predict_proba(X_test)))
 
# Mostrar resumen de metricas    
print(summary_model)

[             precision    recall  f1-score   support      pred       AUC
0             0.588351  0.504878  0.543428   30851.0   26474.0  0.829066
1             0.834045  0.906115  0.868587  346381.0  376312.0  0.807310
2             0.513522  0.384243  0.439574  101506.0   75952.0  0.744981
avg / total   0.750252  0.769607  0.756670  478738.0  478738.0  0.896699,              precision    recall  f1-score   support      pred       AUC
0             0.535371  0.483760  0.508259   30850.0   27876.0  0.825955
1             0.841574  0.914655  0.876594  346381.0  376460.0  0.830065
2             0.540321  0.396042  0.457066  101507.0   74402.0  0.767576
avg / total   0.757968  0.776926  0.763906  478738.0  478738.0  0.903437,              precision    recall  f1-score   support      pred       AUC
0             0.559928  0.492609  0.524116   30850.0   27141.0  0.823617
1             0.837920  0.897434  0.866656  346381.0  370983.0  0.806848
2             0.508032  0.403464  0.449750  1015

In [17]:
from sklearn.metrics import confusion_matrix
confusion = confusion_matrix(y_test, model_final.predict(X_test))

In [18]:
confusion

array([[ 15096,   7379,   8375],
       [  3918, 312242,  30221],
       [  7785,  53424,  40297]], dtype=int64)